In [1]:
import cv2
import numpy as np
import time
import imutils

In [2]:
def color_threshold():
	img = np.zeros((300,300,3), np.uint8)
	cv2.namedWindow("image")

	def value(x):
	    return x

	cv2.createTrackbar("R","image",0,255, value)
	cv2.createTrackbar("G","image",0,255, value)
	cv2.createTrackbar("B","image",0,255, value)

	color_store = []
	count=0
	while (1):
	    cv2.imshow("image", img)
	        
	    r = cv2.getTrackbarPos("R","image")
	    g = cv2.getTrackbarPos("G","image")
	    b = cv2.getTrackbarPos("B","image")
	    
	    img[:] = [b,g,r]
	    
	    k = cv2.waitKey(1) & 0xff
	    if k == ord('s'):
	        if count==0:
	            print("lower_threshold:", (b,g,r))
	            color_store.append([b,g,r])
	            count+=1
	        elif count==1:
	            print("upper_threshold:", (b,g,r))
	            color_store.append([b,g,r])
	            cv2.destroyAllWindows()
	            return color_store
	        
	    if k == ord('q'):
	        print(b,g,r)
	        color_store.append((b,g,r))
	        cv2.destroyAllWindows()
	        break

In [3]:
video = cv2.VideoCapture(0)

color_store = color_threshold()
lowerBlue = np.array(color_store[0], np.uint8)
upperBlue = np.array(color_store[1], np.uint8)

while True:
    ret, frame = video.read()
    frame = imutils.resize(frame, width=800)
    blue = cv2.inRange(frame, lowerBlue, upperBlue)
    blue = cv2.GaussianBlur(blue, (3,3), 0)
    
    (_,cnts,_) = cv2.findContours(blue.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(cnts)>0:
        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[0]
        rects = np.int32(cv2.boxPoints(cv2.minAreaRect(cnts)))
        cv2.drawContours(frame, [rects], -1, (0,255,0), 2)
                    
    cv2.imshow("detection", frame)
    #cv2.imshow("binary", blue)
    
    if cv2.waitKey(1) & 0xff == ord('s'):
        cv2.imwrite("{:5.2f}_pic.jpg".format(time.time()), frame)
    
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
                         
video.release()
cv2.destroyAllWindows()    

lower_threshold: (100, 69, 0)
upper_threshold: (255, 128, 50)
